In [5]:
!pip install implicit -q

In [3]:
!pip install nltk

In [43]:
import random
import numpy as np
import pandas as pd
import implicit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.feature_extraction.text import TfidfVectorizer
import optuna
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score
import nltk
from nltk.corpus import stopwords
import re

import scipy.sparse as sparse
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix

nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
ratings = pd.read_csv('/kaggle/input/recomend-system/ratings.csv')
ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [8]:
books = pd.read_csv('/kaggle/input/recomend-system/books.csv')
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [9]:
# Превращаем рейтинги в бинарные оценки: 1 — понравилось, 0 — не понравилось
ratings['binary_rating'] = ratings['rating'].apply(lambda x: 1 if x >= 4 else 0)

In [10]:
# Нумерация книг для каждого пользователя
ratings['book_order'] = ratings.groupby('user_id').cumcount() + 1
ratings['total_books'] = ratings.groupby('user_id')['book_id'].transform('count')
ratings['book_fraction'] = ratings['book_order'] / ratings['total_books']

In [11]:
ratings.shape

(5976479, 7)

In [12]:
merged_data = pd.merge(ratings, books[['book_id', 'average_rating', 'ratings_count']],
                        on='book_id', how='inner')
merged_data.shape

(5976479, 9)

In [13]:
merged_data.head()

,user_id,book_id,rating,binary_rating,book_order,total_books,book_fraction,average_rating,ratings_count
0,1,258,5,1,1,117,0.008547,4.24,263685
1,2,4081,4,1,1,65,0.015385,3.40,19293
2,2,260,5,1,2,65,0.030769,4.13,282623
3,2,9296,5,1,3,65,0.046154,4.09,9563
4,2,2318,3,0,4,65,0.061538,4.00,43937


In [14]:
train_fraction = 0.7

# Для каждого пользователя книги делятся в зависимости от их порядкового номера
train_data = merged_data[merged_data['book_fraction'] <= train_fraction].copy()
test_data = merged_data[merged_data['book_fraction'] > train_fraction].copy()

# Просмотр тренировочных данных
train_data.head()

,user_id,book_id,rating,binary_rating,book_order,total_books,book_fraction,average_rating,ratings_count
0,1,258,5,1,1,117,0.008547,4.24,263685
1,2,4081,4,1,1,65,0.015385,3.40,19293
2,2,260,5,1,2,65,0.030769,4.13,282623
3,2,9296,5,1,3,65,0.046154,4.09,9563
4,2,2318,3,0,4,65,0.061538,4.00,43937


In [15]:
train_data.shape, test_data.shape

((4159553, 9), (1816926, 9))

In [12]:
# Проверка уникальных пользователей в тренировочном наборе
unique_train_users = train_data['user_id'].unique()
print(f"Количество уникальных пользователей в тренировочном наборе: {len(unique_train_users)}")
print(f"Уникальные пользователи в тренировочном наборе: {unique_train_users}")

# Проверка уникальных пользователей в тестовом наборе
unique_test_users = test_data['user_id'].unique()
print(f"Количество уникальных пользователей в тестовом наборе: {len(unique_test_users)}")
print(f"Уникальные пользователи в тестовом наборе: {unique_test_users}")

# Проверка, есть ли пересечения между тренировочным и тестовым наборами
common_users = set(unique_train_users) & set(unique_test_users)
print(f"Количество общих пользователей в тренировочном и тестовом наборах: {len(common_users)}")

Количество уникальных пользователей в тренировочном наборе: 53424
Уникальные пользователи в тренировочном наборе: [    1     2     4 ... 27329 33111 49802]
Количество уникальных пользователей в тестовом наборе: 53424
Уникальные пользователи в тестовом наборе: [   73    70   100 ... 33111 48801 49802]
Количество общих пользователей в тренировочном и тестовом наборах: 53424


In [16]:
# Создаём матрицу
user_index = train_data.user_id.unique()
books_index = train_data.book_id.unique()


rows = train_data['user_id'].astype(CategoricalDtype(categories=user_index)).cat.codes
cols = train_data['book_id'].astype(CategoricalDtype(categories=books_index)).cat.codes

user_item_matrix = sparse.csr_matrix(
    (train_data.binary_rating, (rows, cols)), shape=(len(user_index), len(books_index))
)

print(user_item_matrix.shape)

(53424, 10000)


In [41]:
def precision_at_k(y_true, y_pred, k=10):
    intersection = np.intersect1d(y_true, y_pred[:k])
    return len(intersection) / k

def rel_at_k(y_true, y_pred, k=10):
    if k <= len(y_pred) and y_pred[k - 1] in y_true:
        return 1
    else:
        return 0

def average_precision_at_k(y_true, y_pred, k=10):
    ap = 0.0
    for i in range(1, k + 1):
        ap += precision_at_k(y_true, y_pred, i) * rel_at_k(y_true, y_pred, i)

    return ap / min(k, len(y_true))

In [79]:
# Создаем и обучаем модель ALS
model = implicit.als.AlternatingLeastSquares(factors=40, iterations=30)

model.fit(user_item_matrix)

  0%|          | 0/30 [00:00<?, ?it/s]

In [81]:
N = 10
test_users = test_data['user_id'].sample(n=N, random_state=42)


In [82]:
all_precisions = []

for user_id in test_users:
    # Получаем истинные книги, которые пользователь положительно оценил
    true_books = list(
        test_data[
            (test_data.user_id == user_id) & (test_data.binary_rating == 1)
        ]['book_id'].values
    )

    # Преобразуем идентификаторы в целые числа
    true_books = [int(book_id) for book_id in true_books]

    # Проверка типов идентификаторов книг в true_books
    incorrect_ids = [book_id for book_id in true_books if not isinstance(book_id, int)]
    if incorrect_ids:
        print("Некорректные идентификаторы в true_books:", incorrect_ids)
    else:
        print("Все идентификаторы книг в true_books - целые числа: True")

    # Получаем книги, которые пользователь уже оценил
    already_seen = set(
        train_data[
            (train_data.user_id == user_id) & (train_data.binary_rating == 1)
        ]['book_id'].values
    )

    # Пропускаем пользователей без истинных оценок
    if len(true_books) == 0:
        continue

    # Получаем рекомендованные книги для пользователя
    recos_books_for_user = []
    recommended_indices = model.recommend(
        user_id,
        user_item_matrix[user_index == user_id],
        N=30,
        filter_already_liked_items=True,
    )[0]

    for ind in recommended_indices:
        if books_index[ind] not in already_seen:
            recos_books_for_user.append(books_index[ind])

    print(f"Пользователь ID: {user_id}")
    print("Истинные книги:", true_books)
    print("Рекомендованные книги:", recos_books_for_user)

    intersection = set(true_books).intersection(set(recos_books_for_user))
    if intersection:
        print("Пересечение:", intersection)
    else:
        print("Нет пересечения между истинными и рекомендованными книгами.")

    # Рассчитываем среднюю точность на уровне k
    ap = average_precision_at_k(
        np.asarray(true_books), np.asarray(recos_books_for_user), k=10
    )

    all_precisions.append(ap)

mean_average_precision = np.mean(all_precisions) if all_precisions else 0
print("Среднее значение mAP@10:", mean_average_precision)

Все идентификаторы книг в true_books - целые числа: True
Пользователь ID: 20221
Истинные книги: [335, 64, 476, 800, 1435, 540, 826, 2824, 379, 1012, 1190, 2088, 34, 85, 98, 140, 285, 143, 267, 4736, 864, 554, 1173, 6831, 1400, 6177, 146, 438, 610, 235, 5481, 2204, 6962, 459, 250, 2772, 2216, 320]
Рекомендованные книги: [50, 19, 29, 5, 58, 8, 28, 3, 278, 339, 296, 79, 549, 157, 62, 53, 101, 89, 125, 59, 454, 154, 661, 71, 627, 364]
Нет пересечения между истинными и рекомендованными книгами.
Все идентификаторы книг в true_books - целые числа: True
Пользователь ID: 6026
Истинные книги: [1275, 551, 1723, 11, 437, 678, 143, 801, 481, 313, 1071, 1536, 335, 267, 1794, 3228, 4148, 482, 2003, 630, 2613, 594, 2060]
Рекомендованные книги: [110, 188, 641, 117, 133, 93, 41, 59, 175, 214, 151, 159, 158, 15, 184, 245, 77, 163, 53, 335, 63]
Пересечение: {335}
Все идентификаторы книг в true_books - целые числа: True
Пользователь ID: 48612
Истинные книги: [8005, 4718, 5217, 3474, 7225, 1637, 6649, 2345,

In [19]:
# Векторизация заголовков книг
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(books.title).astype('float32')

In [20]:
vectorizer.vocabulary_

{'harry': 44,
 'great': 42,
 'girl': 37,
 'fire': 34,
 'lord': 57,
 'blood': 10,
 'chronicles': 16,
 'time': 87,
 'game': 36,
 'ice': 51,
 'love': 59,
 'little': 56,
 'women': 98,
 'life': 55,
 'book': 12,
 'new': 66,
 'moon': 63,
 'city': 17,
 'guide': 43,
 'world': 99,
 'secret': 77,
 'night': 67,
 'dark': 21,
 'saga': 74,
 'glass': 39,
 'tale': 83,
 'trilogy': 88,
 'princess': 70,
 'one': 68,
 'day': 24,
 'things': 85,
 'daughter': 23,
 'club': 18,
 'big': 8,
 'man': 61,
 'sea': 76,
 'home': 48,
 'children': 15,
 'earth': 30,
 'war': 94,
 'story': 81,
 'red': 72,
 'king': 53,
 'lost': 58,
 'american': 5,
 'last': 54,
 'beautiful': 7,
 'dead': 25,
 'vampire': 91,
 'power': 69,
 'art': 6,
 'white': 96,
 'murder': 64,
 'magic': 60,
 'complete': 19,
 'fallen': 32,
 'diaries': 27,
 'history': 47,
 'alex': 4,
 'cross': 20,
 'house': 49,
 'dream': 29,
 'three': 86,
 'school': 75,
 'shadow': 78,
 'girls': 38,
 'never': 65,
 'end': 31,
 'good': 41,
 'jack': 52,
 'universe': 90,
 '11': 1,
 'b

In [21]:
tf_idf_df = pd.DataFrame(X.toarray(), columns=vectorizer.vocabulary_)

In [23]:
books = pd.concat([books, tf_idf_df], axis=1)

In [24]:
vector_columns = list(vectorizer.vocabulary_) + ['book_id']

# Сливаем векторные признаки с train_data 
train_data = train_data.merge(books[vector_columns], on='book_id', how='left')

# Сливаем векторные признаки с test_data 
test_data = test_data.merge(books[vector_columns], on='book_id', how='left')

In [25]:
train_data.shape, test_data.shape

((4159553, 109), (1816926, 109))

In [26]:
# Удаляем ненужный признак для работы модели
train = train_data.drop(columns=['rating'])

test = test_data.drop(columns=['rating'])

In [27]:
# Уменьшаем размер датасета, оставляем 10% выборки
from sklearn.cluster import KMeans

# Разделение данных по целевой переменной
data_majority = train[train['binary_rating'] == 0]
data_minority = train[train['binary_rating'] == 1]

# Применение кластеризации для majority class
kmeans_majority = KMeans(n_clusters=10, random_state=42)
data_majority['cluster'] = kmeans_majority.fit_predict(data_majority)

# Применение кластеризации для minority class
kmeans_minority = KMeans(n_clusters=10, random_state=42)
data_minority['cluster'] = kmeans_minority.fit_predict(data_minority)

# Снижение количества данных для majority class, сохраняя пропорции
reduced_majority = data_majority.groupby('cluster').apply(lambda x: x.sample(frac=0.1)).reset_index(drop=True)

# Снижение количества данных для minority class, сохраняя пропорции
reduced_minority = data_minority.groupby('cluster').apply(lambda x: x.sample(frac=0.1)).reset_index(drop=True)

# Объединение данных
reduced_data = pd.concat([reduced_majority, reduced_minority]).reset_index(drop=True)

In [28]:
# Удаление признака 'cluster'
reduced_data = reduced_data.drop(columns=['cluster'])
reduced_data.shape

(415954, 108)

In [29]:
# Проверяем, сохранился ли первоначальный баланс классов целевой переменной
reduced_data.binary_rating.value_counts(normalize=True)

binary_rating
1    0.693911
0    0.306089
Name: proportion, dtype: float64

In [30]:
# Разделение 
X_train = reduced_data.drop(columns=['binary_rating'])
y_train = reduced_data['binary_rating']

X_test = test.drop(columns=['binary_rating'])
y_test = test['binary_rating']

print(X_train.shape, y_train.shape)
print(X_test.shape,y_test.shape)

(415954, 107) (415954,)
(1816926, 107) (1816926,)


In [31]:
for col in X_train.select_dtypes(include=['int64', 'float64']).columns:
    X_train[col] = X_train[col].astype('float32')


In [32]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415954 entries, 0 to 415953
Columns: 107 entries, user_id to star
dtypes: float32(102), int64(5)
memory usage: 177.7 MB


In [49]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'num_leaves': [8, 16, 32, 64, 128, 256],
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],
    'min_child_samples': [5, 10, 20, 30, 40],
    'reg_alpha': [0.0, 0.01, 0.1, 0.5, 1.0],
    'reg_lambda': [0.0, 0.01, 0.1, 0.5, 1.0],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0]
}

# Создаем объект Random Search
random_search = RandomizedSearchCV(
    LGBMClassifier(random_state=42, boosting_type='gbdt', verbose=-100),
    param_distributions,
    n_iter=50,  
    scoring='average_precision',  
    cv=5,  
    n_jobs=-1,  
    random_state=42
)

# Обучение Random Search
random_search.fit(X_train, y_train)

# Получаем лучшие гиперпараметры
best_params = random_search.best_params_
print(f"Лучшие гиперпараметры: {best_params}")

# Получаем обученную модель
best_model = random_search.best_estimator_

Лучшие гиперпараметры: {'subsample': 0.7, 'reg_lambda': 0.1, 'reg_alpha': 1.0, 'num_leaves': 16, 'n_estimators': 500, 'min_child_samples': 10, 'max_depth': 7, 'learning_rate': 0.001, 'colsample_bytree': 1.0}


In [83]:
# Получаем уникальные user_id
user_ids = ratings['user_id'].unique()

# Создаем словарь для сопоставления user_id с индексами
user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}

recommendations_dict = {}

# Получаем рекомендации для каждого пользователя
num_recommendations = 30

for user_id in user_ids:
    # Получаем индекс пользователя из словаря
    if user_id in user_id_to_index:
        user_index = user_id_to_index[user_id]
        
        # Получаем рекомендованные книги 
        recommended_items = model.recommend(user_index, user_item_matrix[user_index], N=num_recommendations)
        recommendations_dict[user_id] = [item[0] for item in recommended_items]

# Печатаем рекомендации для конкретного пользователя
specific_user_id = 6026  
if specific_user_id in recommendations_dict:
    print(f"Рекомендованные книги для user {specific_user_id}: {recommendations_dict[specific_user_id]}")
else:
    print(f"Не найдено рекомендаций для user {specific_user_id}.")

Рекомендованные книги для user 6026: [3061, 0.46470314]


In [36]:
recommendations_list = []

for user_id, (book_id, probability) in recommendations_dict.items():
    recommendations_list.append({'user_id': user_id, 'book_id': book_id, 'probability': probability})

recommendations_df = pd.DataFrame(recommendations_list)

In [51]:
required_features = best_model.booster_.feature_name()

# Отбираем только необходимые признаки для классификатора
recommended_books_features = recommended_books_features[required_features]

# Применяем классификатор для получения вероятностей
y_pred_proba = best_model.predict_proba(recommended_books_features)

# Добавляем вероятности в DataFrame с рекомендациями
recommended_books_features['probability'] = y_pred_proba[:, 1] 

# Получаем топ-10 рекомендаций для каждого пользователя
top_recommendations = recommended_books_features.groupby('user_id').apply(
    lambda x: x.nlargest(10, 'probability')).reset_index(drop=True)

In [52]:
# Выводим топ-10 рекомендаций для конкретного пользователя
user_recommendations = top_recommendations[top_recommendations['user_id'] == specific_user_id]

if not user_recommendations.empty:
    print(f"Топ-10 рекомендованных книг для пользователя {specific_user_id}:")
    print(user_recommendations[['user_id', 'book_id', 'probability']])
else:
    print(f"Не найдено рекомендаций для пользователя {specific_user_id}.")

Топ-10 рекомендованных книг для пользователя 6026:
      user_id  book_id  probability
6025     6026     5949     0.552367


In [53]:
user_ap_scores = {}

for user_id in top_recommendations['user_id'].unique():
    y_true = test[test['user_id'] == user_id]['book_id'].values
    y_pred = top_recommendations[top_recommendations['user_id'] == user_id]['book_id'].values

    if len(y_true) > 0 and len(y_pred) > 0:
        ap = average_precision_at_k(y_true, y_pred, k=10)
        user_ap_scores[user_id] = ap
        
mAP = np.mean(list(user_ap_scores.values()))

print(f"mAP@10 для гибридных рекомендаций: {mAP}")

mAP@10 для гибридных рекомендаций: 0.0015180443246480984


### С помощью гибридных рекомендаций удалось увеличить метрику mAP@10 с ALS = 0.00111 до классификатор = 0.00151, время на обучение классификатора требуется гораздо больше